In [ ]:
# draw function on 2d elements

import webgpu.jupyter
#from webgpu.mesh import MeshData
from ngsolve import *
import ngsolve_webgpu.jupyter

def render(canvas, mesh, cf, order=2):
    from ngsolve_webgpu.mesh import MeshData, FunctionData, CoefficientFunctionRenderObject, PointNumbersRenderObject, evaluate_cf
    from webgpu.draw import Draw
    import webgpu.jupyter as wj
    import pyodide.ffi
    import ngsolve as ngs
    import js
    from webgpu.scene import Scene, get_scene
    
    mesh_data = MeshData(mesh.ngmesh)
    function_data = FunctionData(mesh_data, cf, order)
    mesh_object = CoefficientFunctionRenderObject(function_data)
    point_numbers = PointNumbersRenderObject(mesh_data, 13)
    Draw([mesh_object, point_numbers], canvas)
    
    mesh_object.colormap.set_min_max(0., 1.)
    
    
m = Mesh(unit_square.GenerateMesh(maxh=.1))
#m = Mesh(unit_cube.GenerateMesh(maxh=.3))
cf=sin(10*x)*sin(10*y)

# "data" will be passed as argument to render on the pyodide side
webgpu.jupyter.DrawCustom(render, kwargs={"cf": cf, "mesh": m, "order": 5}, modules=["ngsolve_webgpu"])


In [ ]:
# draw geometry

from netgen.occ import *
from ngsolve import *
import webgpu.jupyter
import ngsolve_webgpu.jupyter

box = Box((-1,-1,-1), (1,1,1)) + Sphere((1,1,1), 0.4)

def setup(canvas, geo):
    import ngsolve_webgpu.geometry
    from webgpu.scene import Scene
    from webgpu.draw import Draw
    obj = ngsolve_webgpu.geometry.GeometryRenderObject(geo)
    Draw(obj, canvas)

def draw_geo(geo):
    if isinstance(geo, TopoDS_Shape):
        geo = OCCGeometry(geo)
    webgpu.jupyter.DrawCustom(setup, kwargs={"geo" : geo}, modules=["ngsolve_webgpu"])

draw_geo(box)

In [ ]:
# draw isosurface
import webgpu.jupyter
import ngsolve_webgpu.jupyter
from netgen.occ import *
from ngsolve import *

box = Box((-1,-1,-1), (1,1,1))
geo = OCCGeometry(box)
mesh = Mesh(geo.GenerateMesh(maxh=.15))
gf = GridFunction(H1(mesh, order=3))
cf = x**2 + y**2 + z**2 - 0.5**2
gf.Set(cf)

def setup(canvas, levelset, func, mesh):
    import ngsolve_webgpu.isosurface
    from webgpu.draw import Draw
    Draw(ngsolve_webgpu.isosurface.IsoSurfaceRenderObject(levelset, func, mesh, "Isosurface"), canvas)

def draw_iso(levelset, func, mesh):
    import ngsolve_webgpu.isosurface
    iso = ngsolve_webgpu.isosurface.IsoSurfaceRenderObject(levelset, func, mesh, "Isosurface")
    scene = webgpu.jupyter.Scene([iso])
    webgpu.jupyter.Draw(scene, modules=["ngsolve_webgpu"])
    return scene

scene = draw_iso(gf, x/Norm(CF((x,y,z))), mesh)

In [ ]:
gf.Set(x**2 + y**2 + z**2 - 0.7**2)
scene.redraw()